In [1]:
import pandas as pd
import numpy as np
from joblib import dump
from sklearn.model_selection import train_test_split
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [2]:
df = pd.read_csv("foods_lang.csv", low_memory=False)

In [3]:
df = df[df["lang"] == "en"]

In [4]:
#df.isna().sum()

In [5]:
df = df[["ingredients_text", "additives_n", "nutriscore_score", "nutriscore_grade", "fat_100g", "saturated-fat_100g", "carbohydrates_100g", "sugars_100g", "proteins_100g", "salt_100g"]]

In [6]:
df.dropna(subset=["fat_100g", "saturated-fat_100g", "carbohydrates_100g", "sugars_100g", "proteins_100g", "salt_100g"], inplace=True)

In [8]:
fats = [float(x) for x in df["fat_100g"]]
carbs = [float(x) for x in df["carbohydrates_100g"]]
proteins = [float(x) for x in df["proteins_100g"]]

In [9]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(['a', 'b', 'c', 'd', 'e'])
grade = le.transform(df["nutriscore_grade"])

le = preprocessing.LabelEncoder()
le.fit(list(set(df["nutriscore_score"])))
score = le.transform(df["nutriscore_score"])

In [7]:
import re
def extract_ingredients(text):
    text = re.split(", | \(|\)|\[|\]", text)
    text = [x.replace("from ", "") for x in text]
    text = [x.replace("_", "") for x in text]
    text = [re.sub(r'\d+\.\d+%', "", x) for x in text]
    text = [re.sub(r'\d+\,\d+%', "", x) for x in text]
    text = [re.sub(r'\d+%', "", x) for x in text]
    text = [re.sub("^\s+|\s+$", "", x, flags=re.UNICODE) for x in text]
    text = [x.replace(".", "") for x in text]
    text = [x.replace("fd&c ", "") for x in text]
    text = [x for x in text if len(x) > 1]
    text = [x.lower() for x in text]
    return text

In [8]:
products = []
for x in list(df["ingredients_text"]):
     products.append(extract_ingredients(x))

In [9]:
ingredients = []
for p in products:
    ingredients += p

In [11]:
Counter(df["nutriscore_grade"])

2     c
20    c
21    d
22    d
24    d
25    d
48    d
49    d
50    c
51    c
Name: nutriscore_grade, dtype: object

In [13]:
le_ingr = preprocessing.LabelEncoder()
le_ingr.fit(list(set(ingredients)))

LabelEncoder()

In [14]:
%%time
ingredients = le_ingr.transform(ingredients)

Wall time: 13.6 s


In [15]:
%%time
start = 0
products_encoded = []
for x in products:
    products_encoded.append(ingredients[start:start+len(x)])
    start += len(x)

Wall time: 187 ms


In [16]:
dump(le_ingr, "processed_data/ingredients_encoder.joblib")

['processed_data/ingredients_encoder.joblib']

In [17]:
with open("processed_data/ingredients.npy", "wb") as f:
    np.save(f, np.array(ingredients, dtype=object))

with open("processed_data/products.npy", "wb") as f:
    np.save(f, np.array(products_encoded, dtype=object))

In [18]:
with open("processed_data/grade.npy", "wb") as f:
    np.save(f, np.array(score))

with open("processed_data/score.npy", "wb") as f:
    np.save(f, np.array(grade))

In [19]:
with open("processed_data/fats.npy", "wb") as f:
    np.save(f, np.array(fats))

with open("processed_data/carbs.npy", "wb") as f:
    np.save(f, np.array(carbs))

with open("processed_data/proteins.npy", "wb") as f:
    np.save(f, np.array(proteins))